## Description

#### Purpose: To get MPAA ratings from TMDB.

#### Input: `3.3.8_Merged_Data_Dist_Scores.csv`

#### Outputs: `3.3.9a_Merged_Data_MPAA.csv`

This notebook retrieves MPAA ratings from TMDB, which was not initially retrieved in step 2.1 Collect Initial Data.

In [ ]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
from tmdbv3api.exceptions import TMDbException
import random
import pandas as pd
import csv
tmdb=TMDb()
tmdb.api_key=' '
    # API key redacted
tmdb.language='en'
movie=Movie()

In [ ]:
full_data = pd.read_csv('../3.3.8 Calculate Distributor Score/Outputs/3.3.8_Merged_Data_Dist_Scores.csv', header=0, index_col=0)
tmdb_ids = full_data['TMDB ID']

mpaa = {}
for id in tmdb_ids:
    try:
        int_id = int(id)
        movie_info=movie.details(int_id)
        if movie_info:
            release_dates=movie_info.get('release_dates',{})
            results=release_dates.get('results',[])

            for result in results:
            # THIS (BELOW) IS THE LINE ADDED FOR US RELEASE DATE ONLY
                if result.get('iso_3166_1')=='US':
                    for release_date in result.get('release_dates',[]):
                        if release_date.get('type')==3:
                    #Write the movie ID to the list
                            mpaa[int_id] = release_date.get('certification')
    except:
        pass

In [ ]:
mpaa_df = pd.DataFrame({'TMDB ID': list(mpaa.keys()), 'MPAA_TMDB': list(mpaa.values())})
full_data_mpaa = full_data.merge(mpaa_df, 'left', left_on='TMDB ID', right_on='TMDB ID')
full_data_mpaa.to_csv('/Outputs/3.3.9a_Merged_Data_MPAA.csv')